# Westeros Tutorial - Firm capacity

In the previous part, we showed how to introduce emissions into a stylized energy systems model, and what happens if you put a constraint on total CO2 emissions.

In this tutorial notebook, we will illustrate how to add other considerations, in particular the requirement to have sufficient dispatchable (firm) capacity.

In [ ]:
import pandas as pd
import ixmp
import message_ix

from message_ix.utils import make_df

%matplotlib inline

In [ ]:
mp = ixmp.Platform(dbtype='HSQLDB')

In [ ]:
base = message_ix.Scenario(mp, model='Westeros Electrified', scenario='emission_bound')

In [ ]:
model = 'Westeros Electrified'
scen = base.clone(model, 'firm_capacity','illustration of firm-capacity formulation',
                  keep_solution=False)
scen.check_out()

year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']
model_horizon = scen.set('year')
country = 'Westeros'

## Add Renewable Formulation

### Load Factors - Describing the Electricity Sector
#### Peak load 
If we further specify our demand, we will find that the demand for electricity (!not the demand for useful light but for electricity!) has a peak load that is different from the annual average load; e.g: if annual average load is 5 GWh peak load during midday might be 10GW --> the peak load factor is equal to 2.

In [ ]:
# peak_load_factor(node,commodity,level,year,time)
peak_load_factor = pd.DataFrame({
        'node': country,
        'commodity': 'electricity',
        'level' : 'secondary',       
        'year': model_horizon,
        'time' : 'year',
        'value' : 2,
        'unit' : '???'})

scen.add_par('peak_load_factor', peak_load_factor)

## Describing the Renewable Technologies Reliability
### rating and reliability --> Ensuring enough Firm Capacity
we assume, that up to a share of 20% of total electricity supply, the capacity of wind_ppl adds by 80% to the firm capacity of the power system (r1). The remaining 90% installed capacity, however  only  5% contribute to the firm capcity (r2). The coal power plant supplies full reliability independet of the supply share.


In [ ]:
base_reliability = pd.DataFrame({
        'node': country,
        'commodity': 'electricity',
        'level' : 'secondary', 
        'unit': '???',
        'time': 'year',
        'year_act': model_horizon})

In [ ]:
# add the ratings as a set 
scen.add_set('rating', ['r1', 'r2'])

# for Wind PPL
rating_bin = make_df(base_reliability, technology= 'wind_ppl', value = 0.5, rating= 'r1')
scen.add_par('rating_bin', rating_bin)

reliability_factor = make_df(base_reliability, technology= 'wind_ppl', value = 0.5, rating= 'r1')
scen.add_par('reliability_factor', reliability_factor)

rating_bin = make_df(base_reliability, technology= 'wind_ppl', value = 0.5, rating= 'r2')
scen.add_par('rating_bin', rating_bin)

reliability_factor = make_df(base_reliability, technology= 'wind_ppl', value = 0.25, rating= 'r2')
scen.add_par('reliability_factor', reliability_factor)

# for Coal PPL
reliability_factor = make_df(base_reliability, technology= 'coal_ppl', value = 1, rating= 'firm')
scen.add_par('reliability_factor', reliability_factor)

## Commit and Solve

In [ ]:
scen.commit(comment='define parameters for renewable implementation')
scen.set_as_default()

In [ ]:
scen.solve()

In [ ]:
scen.var('OBJ')['lvl']

## Plotting

In [ ]:
from tools import Plots
p = Plots(scen, country, firstyear=700)

In [ ]:
p.plot_activity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

In [ ]:
p.plot_capacity(baseyear=True, subset=['coal_ppl', 'wind_ppl'])

In [ ]:
p.plot_prices(subset=['light'], baseyear=True)

## Close the connection to the database

In [ ]:
mp.close_db()